# Global Equity Portfolio Optimization and External Manager Allocation

## Introduction

The Global Equity portfolio invested in 5 external managers and 2 subportfolios managed internally. The portfolio is designed to generate long term investment return while maintain moderate risk compared to benchmark MSCI World Index.

During the past years, the performance of the portfolio is not perfoming as expected. To better understand how to allocate capital in different fund managers, I perform the portfolio opimization technique to get the highest possible returns, sharp ratios, with given risk. 

Based on the opitmal weights, we backtest the performance on different weighted portfolios. We found that the current portfolio is not the best performed one.

Notice that the estimated return is based on history data which could be challenged.



In [5]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize


def equal_weight(assets):
    optimal = [1/len(assets) for i in range(len(assets))]
    return optimal


def minimum_variance(ret, bound):
    def find_port_variance(weights):
        # this is actually std
        cov = ret.cov()
        port_var = np.sqrt(np.dot(weights.T, np.dot(cov, weights)) * 250)
        return port_var

    def weight_cons(weights):
        return np.sum(weights) - 1


    bounds_lim = [bound for x in range(len(ret.columns))] # change to (-1, 1) if you want to short
    init = [1/len(ret.columns) for i in range(len(ret.columns))]
    constraint = {'type': 'eq', 'fun': weight_cons}

    optimal = minimize(fun=find_port_variance,
                       x0=init,
                       bounds=bounds_lim,
                       constraints=constraint,
                       method='SLSQP'
                       )

    return list(optimal['x'])


def max_sharpe(ret, bound):
    def sharpe_func(weights):
        hist_mean = ret.mean(axis=0).to_frame()
        hist_cov = ret.cov()

        port_ret = np.dot(weights.T, hist_mean.values) * 250
        port_std = np.sqrt(np.dot(weights.T, np.dot(hist_cov, weights)) * 250)
        return -1 * port_ret / port_std

    def weight_cons(weights):
        return np.sum(weights) - 1


    bounds_lim = [bound for x in range(len(ret.columns))] # change to (-1, 1) if you want to short
    init = [1/len(ret.columns) for i in range(len(ret.columns))]
    constraint = {'type': 'eq', 'fun': weight_cons}

    optimal = minimize(fun=sharpe_func,
                       x0=init,
                       bounds=bounds_lim,
                       constraints=constraint,
                       method='SLSQP'
                       )

    return list(optimal['x'])

def minimum_trackingerror(ret, benchmark_ret, bound):
    def tracking_error(weights):
        tracking_difference = ret - benchmark_ret
        cov = tracking_difference.cov()
        tracking_error = np.sqrt(np.dot(weights.T, np.dot(cov, weights))*250)        
        return tracking_error

    def weight_cons(weights):
        return np.sum(weights)-1

    
    bounds_lim = [bound for x in range(len(ret.columns))] # change to (-1, 1) if you want to short
    init = [1/len(ret.columns) for i in range(len(ret.columns))]
    constraint = {'type': 'eq', 'fun': weight_cons}

    optimal = minimize(fun=tracking_error,
                       x0=init,
                       bounds=bounds_lim,
                       constraints=constraint,
                       method='SLSQP'
                       )

    return list(optimal['x'])